### LSB steganography method

In [4]:
### LSB Encryption method

import numpy as np 
import random 
  
# Encryption function 
def encrypt(cover, secret, output): 
      
    # img1 and img2 are the 
    # two input images 
    img1 = cv2.imread(cover) 
    img2 = cv2.imread(secret) 
      
    for i in range(img2.shape[0]): 
        for j in range(img2.shape[1]): 
            for l in range(3): 
                  
                # v1 and v2 are 8-bit pixel values 
                # of img1 and img2 respectively 
                v1 = format(img1[i][j][l], '08b') 
                v2 = format(img2[i][j][l], '08b') 
                  
                # Taking 4 MSBs of each image 
                v3 = v1[:4] + v2[:4]  
                  
                img1[i][j][l]= int(v3, 2) 
                  
    cv2.imwrite(output, img1) 
  
      
# Decryption function 
def decrypt(inp, out1, out2): 
      
    # Encrypted image 
    img = cv2.imread(inp)  
    width = img.shape[0] 
    height = img.shape[1] 
      
    # img1 and img2 are two blank images 
    img1 = np.zeros((width, height, 3), np.uint8) 
    img2 = np.zeros((width, height, 3), np.uint8) 
      
    for i in range(width): 
        for j in range(height): 
            for l in range(3): 
                v1 = format(img[i][j][l], '08b') 
                v2 = v1[:4] + chr(random.randint(0, 1)+48) * 4
                v3 = v1[4:] + chr(random.randint(0, 1)+48) * 4
                  
                # Appending data to img1 and img2 
                img1[i][j][l]= int(v2, 2) 
                img2[i][j][l]= int(v3, 2) 
      
    # These are two images produced from 
    # the encrypted image 
    cv2.imwrite(out1, img1) 
    cv2.imwrite(out2, img2) 

In [5]:
# Test encryption
encrypt('test.jpg', 'secret.png', 'out.png') 

In [ ]:
# Test decryption
decrypt('out.png', 're.png', 'im.png') 

### Image DSP analysis

In [1]:
### DFT analysis
# DSP Library for python
import scipy.signal
import numpy as np 
import cv2

In [2]:
test = cv2.imread('test.jpg') 
test.shape

(1080, 1920, 3)

1080